In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/keras-facenet/model/facenet_keras.h5')

print(model.inputs)
print(model.outputs)

In [ ]:
pip install mtcnn

In [ ]:
pip install keras_facenet

In [ ]:
from fileinput import filename
import os
from os import listdir
import scipy
import cv2
import dlib 
import numpy as np
from PIL import Image # 이미지 크기 조정 라이브러리
import matplotlib.pyplot as plt
import tensorflow as tf 
from mtcnn.mtcnn import MTCNN #불러온 사진에 얼굴 감자히는 '얼굴 감지기'라이브러리
import keras
from keras.models import load_model
from keras_facenet import FaceNet
from tensorflow.keras.models import Model
from numpy import savez_compressed

In [ ]:
def extract_face(filename, required_size=(160, 160)):
	image = Image.open(filename)
	image = image.convert('RGB')
	pixels = np.asarray(image)# 배열로 변환
	detector = MTCNN()# 감지기 생성, 기본 가중치 이용
	results = detector.detect_faces(pixels)# 이미지에서 얼굴 감지
	x1, y1, width, height = results[0]['box']# 첫 번째 얼굴에서 경계 상자 추출
	# 버그 수정
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# 얼굴 추출
	face = pixels[y1:y2, x1:x2]
	# 모델 사이즈로 픽셀 재조정
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = np.asarray(image)
	return face_array

In [ ]:
folder = '/content/drive/MyDrive/facedataset/train/'
i = 1

In [ ]:
# 디렉토리 안의 모든 이미지를 불러오고 이미지에서 얼굴 추출
def load_faces(directory):
	faces = list()
	# 파일 열거
	for filename in listdir(directory):
		# 경로
		path = directory + filename
		# 얼굴 추출
		face = extract_face(path)
		# 저장
		faces.append(face)
	return faces

In [ ]:
# 이미지를 포함하는 각 클래스에 대해 하나의 하위 디렉토리가 포함된 데이터셋을 불러오기
def load_dataset(directory):
	X, y = list(), list()
	# 클래스별로 폴더 열거
	for subdir in listdir(directory):
		# 경로
		path = directory + subdir + '/'
		# 디렉토리에 있을 수 있는 파일을 건너뛰기(디렉토리가 아닌 파일)
		if not os.path.isdir(path):
			continue
		# 하위 디렉토리의 모든 얼굴 불러오기
		faces = load_faces(path)
		# 레이블 생성
		labels = [subdir for _ in range(len(faces))]
		# 진행 상황 요약
		print('>%d개의 예제를 불러왔습니다. 클래스명: %s' % (len(faces), subdir))
		# 저장
		X.extend(faces)
		y.extend(labels)
	return np.asarray(X), np.asarray(y)

In [ ]:
# 훈련 데이터셋 불러오기
trainX, trainy = load_dataset('/content/drive/MyDrive/facedataset/train/')
print(trainX.shape, trainy.shape)
# 테스트 데이터셋 불러오기
testX, testy = load_dataset('/content/drive/MyDrive/facedataset/test/')
print(testX.shape, testy.shape)
# 배열을 단일 압축 포맷 파일로 저장
np.savez_compressed('3-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

data = np.load('3-celebrity-faces-dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('불러오기: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

In [ ]:
model.summary()

In [ ]:
from numpy import expand_dims
# 하나의 얼굴의 얼굴 임베딩 얻기
def get_embedding(model, face_pixels):
	# 픽셀 값의 척도
	face_pixels = face_pixels.astype('int32')
	# 채널 간 픽셀값 표준화(전역에 걸쳐)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# 얼굴을 하나의 샘플로 변환
	samples = expand_dims(face_pixels, axis=0)
	# 임베딩을 갖기 위한 예측 생성
	yhat = model.predict(samples)
	return yhat[0]

In [ ]:
# 훈련 셋에서 각 얼굴을 임베딩으로 변환하기
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = np.asarray(newTrainX)
print(newTrainX.shape)
# 테스트 셋에서 각 얼굴을 임베딩으로 변환하기
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = np.asarray(newTestX)
print(newTestX.shape)
# 배열을 하나의 압축 포맷 파일로 저장
np.savez_compressed('3-celebrity-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

In [ ]:
from sklearn import preprocessing# 3명의 유명인사 얼굴 데이터셋으로 분류기 개발
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

# 데이터셋 불러오기
data = load('/content/3-celebrity-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('데이터셋: 훈련 %d개, 테스트 %d개' % (trainX.shape[0], testX.shape[0]))
# 입력 벡터 일반화
in_encoder = Normalizer(norm='l2') #유클리디안 거리=ㅣ2
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# 목표 레이블 암호화
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# 모델 맞추기(적합시키기)
classifier= svm.SVC(kernel='linear', probability=True)
classifier.fit(trainX, trainy)
# 유클리디안으로 거리로 찾을땐 정확도가 높아지는지 확인
# 추측
yhat_train = classifier.predict(trainX)
yhat_test = classifier.predict(testX)
# 정확도 점수
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# 요약
print('정확도: 훈련=%.3f, 테스트=%.3f' % (score_train*100, score_test*100))

In [ ]:
# 얼굴 불러오기
data = load('3-celebrity-faces-dataset.npz')
testX_faces = data['arr_2']

# 테스트 데이터셋에서 임의의 예제에 대한 테스트 모델
from random import choice
selection = choice([i for i in range(testX.shape[0])])
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])
# 얼굴 예측
samples = expand_dims(random_face_emb, axis=0)
yhat_class = classifier.predict(samples)
yhat_prob = classifier.predict_proba(samples)
# 이름 얻기
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('예상: %s (%.3f)' % (predict_names[0], class_probability))
print('추측: %s' % random_face_name[0])
# 재미삼아 그리기
plt.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
plt.title(title)
plt.show()

In [ ]:
import joblib
joblib.dump(classifier, './classifier.pkl')

In [ ]:
joblib.dump(out_encoder, './out_encoder.pkl')

In [ ]:
from flask import Flask

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/keras-facenet/model/facenet_keras.h5')
classifier = joblib.load('/content/classifier.pkl')
out_encoder = joblib.load('/content/out_encoder.pkl')

In [ ]:
# from flask_ngrok import run_with_ngrok
# app = Flask(__name__)
# run_with_ngrok(app)
# @app.route('/', method=['POST']
# def predict():
#   if request.method == 'POST':
#       file = requests.files.get('/content/drive/MyDrive/facedataset/test/김희애/김희애test.jpg', '')
#       img_bytes = file.read()
#       img = io.BytesIO(img_bytes)

#       # 얼굴 영역 추출
#       face = extract_face(img)
#       # 임베딩벡터로 변환
#       embedding = get_embedding(model, face_pixels)

#       # classifier입력을 위한 전처리
#       embedding = preprocess_something(embedding)

#       # 결과
#       output = dict()
#       yhat = classifier.predict(embedding)
#       output['label'] = out_encoder.inverse_transform(yhat)[0]

#       output['prob'] = classifier.predict_prob(embedding)

#       ret_img = draw_result(img)
#       ret_bytes = io.BytesIO()
#       ret_img.save(ret_bytes, format='PNG')
#       output['retImg'] = base64.encodebytes(ret_bytes.getvalue()).decode('ascii')
#     return jsonify(output)

In [ ]:
# !pip install flask-ngrok
# from flask import Flask, url_for, redirect, render_template, request
# from flask_ngrok import run_with_ngrok
# import os

# app = Flask(__name__)
# run_with_ngrok(app)   #starts ngrok when the app is run

# @app.route('/')
# def index():
#     return render_template('index.html')  # Start ngrok when app is run

# if __name__ == '__main__':
#     app.run()